In [1]:
import pickle, h5py, os
import numpy as np


In [2]:
%load_ext autoreload
%autoreload 2
import utils as ut

Laoding the labels of the data

In [3]:
pickle_out = open('labels_subjects.pkl',"rb")
temp = pickle.load(pickle_out)
labels = np.array(temp[0])

In [4]:
#initalizations
filename = 'All_activity_instances_data.h5'
path_to_data = # Data path of the created dictionary

# dataset information and source activities 
total_subjects = np.unique(temp[1]).shape[0] 
total_Activities = 33
num_source_activities =  int(np.round(0.8*total_Activities)) # number of activities used for training source network
num_target_activities = total_Activities - num_source_activities # number of activities used for traget network

In [5]:
# Loading the osurce and target classes
pickle_out = open('source_activities.pkl',"rb")
src_act_array = pickle.load(pickle_out)
trg_act_array = np.arange(total_Activities)[~np.isin(np.arange(total_Activities),src_act_array)]

Loading the dataset

# Making the Training and Validation set

In [56]:
from sklearn.utils import resample


samples_per_class_train = 300
samples_per_class_Val = 20 # it is assumed that the class samples are greater than this number


total_samples_train = samples_per_class_train *src_act_array.shape[0]
indices_array_train = np.zeros(total_samples_train)
new_labels_train = np.zeros(total_samples_train)

total_samples_Val = samples_per_class_Val *src_act_array.shape[0]
indices_array_Val = np.zeros(total_samples_Val)
new_labels_Val = np.zeros(total_samples_Val)



start = 0
end = start+ samples_per_class_train
startv= 0 
endv = start + samples_per_class_Val


for i in range(src_act_array.shape[0]):
    temp = np.where(labels == src_act_array[i])[0] # these are all the indices of that class
    val_temp = np.random.choice(temp,samples_per_class_Val,replace = False)#now choose sample equal to sampel_per_class_Val
    train_temp = temp[~np.isin(temp,val_temp)]
    
    if(train_temp.shape[0]<samples_per_class_train):
        replace_bool = True
    else:
        replace_bool = False
    indices_resampled = resample(train_temp,replace=replace_bool,n_samples=samples_per_class_train,random_state=42)
    indices_array_train[start:end] = indices_resampled
    new_labels_train[start:end] = src_act_array[i] 
    
    indices_array_Val[startv:endv] = val_temp
    new_labels_Val[startv:endv] = src_act_array[i] 
    
    startv = endv
    endv = startv+samples_per_class_Val
    start = end
    end = start+samples_per_class_train
    
indices_array_Val,new_labels_Val = ut.shuffle_indices(indices_array_Val,new_labels_Val)
indices_array_train,new_labels_train = ut.shuffle_indices(indices_array_train,new_labels_train)


In [57]:
with open('Indices_Train_Val.pkl', 'wb') as handle:
    pickle.dump([indices_array_train,indices_array_Val,new_labels_train,new_labels_Val], handle)

# Target Classes

Select target support set and test set

In [28]:
# Note: The validation set is actually the support set and train here is the test set for the target.Just resusing the code so
# not changing the notation

total = 0
for i in range(trg_act_array.shape[0]):
    temp = np.where(labels == trg_act_array[i])[0] # these are all the indices of that class
    total = total+temp.shape[0]

samples_per_class_Val = 5 # it is assumed that the class samples are greater than this number
total_samples_Val = samples_per_class_Val *trg_act_array.shape[0]
indices_array_Val = np.zeros(total_samples_Val)
new_labels_Val = np.zeros(total_samples_Val)

 

total_samples_train = total - total_samples_Val
indices_array_train = np.zeros(total_samples_train)
new_labels_train = np.zeros(total_samples_train)





start = 0
startv= 0 
endv = startv+ samples_per_class_Val


for i in range(trg_act_array.shape[0]):

    temp = np.where(labels == trg_act_array[i])[0] # these are all the indices of that class
    val_temp = np.random.choice(temp,samples_per_class_Val,replace = False)
    train_temp = temp[~np.isin(temp,val_temp)]


    end = start+train_temp.shape[0]
#     print(i,start,end,train_temp.shape[0])
    indices_array_train[start:end] = train_temp[:]
    new_labels_train[start:end] = trg_act_array[i] 

    indices_array_Val[startv:endv] = val_temp
    new_labels_Val[startv:endv] = trg_act_array[i] 

    startv = endv
    endv = startv+samples_per_class_Val
    start = end


In [29]:
indices_array_supp,new_labels_supp = ut.shuffle_indices(indices_array_Val,new_labels_Val)
indices_array_test,new_labels_test = ut.shuffle_indices(indices_array_train,new_labels_train)

In [30]:
with open('Indices_trg_Test_supp.pkl', 'wb') as handle:
    pickle.dump([indices_array_test,indices_array_supp,new_labels_test,new_labels_supp], handle)